In [ ]:
import os
import ase.io
import re
import pandas as pd
import numpy as np
from dscribe.descriptors import SOAP
from dscribe.descriptors import MBTR,ACSF,CoulombMatrix,EwaldSumMatrix,SineMatrix 
import tensorflow as tf
from tensorflow.keras.models import Model
from numpy                 import array
from sklearn               import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models          import Sequential
from keras.layers          import *
from sklearn               import metrics
from tensorflow.keras import layers
from tcn import TCN
import keras
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Conv2D, MaxPool2D,LSTM,Bidirectional
from keras.models import Sequential
from keras import Input
from keras.layers          import *
from keras.models import save_model,load_model
#####decribe parameters#####
rcut = 6
#####decribe parameters#####
sm = SineMatrix(
    n_atoms_max=96,
    permutation="sorted_l2",
    sparse=False,
    flatten=False)
ew=EwaldSumMatrix(
    n_atoms_max=96,
    permutation="none",flatten=False,sparse=False)
acsf=ACSF(species=['Na','Mg','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Nb','Mo','Tc','Rh','Pd','Ag','Cd','Cs','Ba','Os','Re','Ir','Pt','Au','Ga', 'In', 'Sn', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Hg', 'Tl', 'Pb', 'Bi', 'Li', 'Al','Ru','O'],
# acsf=ACSF(species=['Na','Mg','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Nb','Mo','Tc','Rh','Pd','Ag','Cd','Cs','Ba','Os','Re','Ir','Pt','Au','Ru','O'],
    rcut=rcut,
    periodic=True,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]])

In [ ]:
def find_energy(content):
    number=re.findall('\d+',content[-1])
    splitted=content[-1].split()
    energy=float(splitted[4])
    print(energy)
    return energy
def read_energy(dir_list,describer1,describer2,describer3):
    list_input=[]
    list_energy_SLAB=[]
    list_energy_O=[]
    list_energy_OH=[]
    list_energy_OOH=[]
    list_energy_OVP=[]
    list_not_needed_dir=[]
    i=0
    for dir_each in dir_list:
        try:
            dir_each=path+dir_each
            Read_Struct_SLAB=ase.io.read(dir_each+'/POSCAR_INIT')
            des1=describer1.create(Read_Struct_SLAB,n_jobs=-1)
            des2=describer2.create(Read_Struct_SLAB,n_jobs=-1)
            des3=describer3.create(Read_Struct_SLAB,n_jobs=-1)
            new_des=np.concatenate((des1,des2),axis=1)
            new_des=np.concatenate((new_des,des3),axis=1)
            
            print(des1.shape,des2.shape,des3.shape,new_des.shape)
            new_des=new_des.reshape(768,969)
            sub_dir_number=os.listdir(dir_each+'/ADS_O')
            with open(dir_each+'/OSZICAR',"r") as SLAB:
                content_SLAB=SLAB.read().splitlines()
            sub_dir_number=os.listdir(dir_each+'/ADS_O')
            with open(dir_each+'/ADS_O/'+sub_dir_number[0]+'/OSZICAR',"r") as O:
                content_O=O.read().splitlines()
            with open(dir_each+'/ADS_OH/'+sub_dir_number[0]+'/OSZICAR',"r") as OH:
                content_OH=OH.read().splitlines()
            with open(dir_each+'/ADS_OOH/'+sub_dir_number[0]+'/OSZICAR',"r") as OOH:
                content_OOH=OOH.read().splitlines()
            #####################################
            energy_SLAB=find_energy(content_SLAB)
            energy_OOH=find_energy(content_OOH)
            energy_OH=find_energy(content_OH)
            energy_O=find_energy(content_O)
            print(energy_OH)
            #####################################
            A=(energy_O+0.05)+1*(-6.8)-energy_SLAB-(-14.22)
            B=(energy_OH+0.34)+0.5*(-6.8)-energy_SLAB-(-14.22)
            C=(energy_OOH+0.37)+1.5*(-6.8)-energy_SLAB-2*(-14.22)
            #####################################
            STEP1=B
            STEP2=A-B
            STEP3=C-A
            STEP4=4.92-C
            energy_list_4steps=[STEP1,STEP2,STEP3,STEP4]
            #####################################
            OVP=max(energy_list_4steps)-1.23
            ############whether or not to record this one###########
            if 0.32<OVP<=2.52:
                list_input.append(new_des)
                list_energy_SLAB.append(energy_SLAB)
                list_energy_O.append(energy_O)
                list_energy_OH.append(energy_OH)
                list_energy_OOH.append(energy_OOH)
                list_energy_OVP.append(OVP)
            else:
                print(dir_each)
                list_not_needed_dir.append(dir_each)
            i+=1
            print(i)
            print(list_energy_OH)
        except Exception as e:
            print('folder not exist')
            print(e)
    return list_input,list_energy_SLAB,list_energy_O,list_energy_OH,list_energy_OOH,list_energy_OVP

In [ ]:
path="./result_DFT_B/"
dir_list=os.listdir(path)
list_input,list_energy_SLAB,list_energy_O,list_energy_OH,list_energy_OOH,list_energy_OVP=read_energy(dir_list,sm,ew,acsf)

In [ ]:
# specify the input and output######
######specify the input and output######
data_input=np.around(list_input,decimals=8)
data_output_SLAB=np.around(list_energy_SLAB,decimals=8)
data_output_O=np.around(list_energy_O,decimals=8)
data_output_OH=np.around(list_energy_OH,decimals=8)
data_output_OOH=np.around(list_energy_OOH,decimals=8)
data_output_OVP=np.around(list_energy_OVP,decimals=8)

In [ ]:
data_input.shape

In [ ]:
from numpy import save
np.save('data_input_sm_ew_acsf_rcut6_B.npy',data_input)
np.save('data_output_sm_ew_acsf_rcut6_SLAB_B.npy',data_output_SLAB)
np.save('data_output_sm_ew_acsf_rcut6_O_B.npy',data_output_O)
np.save('data_output_sm_ew_acsf_rcut6_OH_B.npy',data_output_OH)
np.save('data_output_sm_ew_acsf_rcut6_OOH_B.npy',data_output_OOH)
np.save('data_output_sm_ew_acsf_rcut6_OVP_B.npy',data_output_OVP)